<a href="https://colab.research.google.com/github/opsabarsec/NLP--film-genres-from-synopsis/blob/main/radix_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic assignment of genres from movie synopsis using supervised machine learning

## 1. Import libraries and load data

In [24]:
#packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# NLP libraries

from textblob import TextBlob, Word
import nltk
import re
import csv
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

# Deep learning libraries

from keras.models import Model
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, Input, LSTM, GlobalMaxPool1D, Dropout
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

In [25]:
# load data
train= pd.read_csv('./data/train.csv')
test=pd.read_csv('./data/test.csv')

## 2. DATA PREPARATION 

In [26]:
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
# function for text cleaning 
def preprocess_text(text): #funsi yang berguna untuk  melakukan cleaning terhadap dataset
    text = text.lower() # lowercase
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text) #line breaks
    #text = re.sub(r"\'\xa0", " ", text) # xa0 Unicode representing spaces
    #text = re.sub('\s+', ' ', text) # one or more whitespace characters
    text = text.strip(' ') # spaces
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    #lemmatize and remove stopwords
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    text = ' '.join(no_stopword_text) 
        
    return text

train['clean_plot'] = train['synopsis'].apply(lambda x: preprocess_text(x))
test['clean_plot'] = test['synopsis'].apply(lambda x: preprocess_text(x))

In [28]:
def lemma(text): # Lemmatization of cleaned body (avoid redundancy and improve the accuracy of analysis.)
        sent = TextBlob(text)
        tag_dict = {"J": 'a', 
                    "N": 'n', 
                    "V": 'v', 
                    "R": 'r'}
        words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
        lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
        seperator=' '
        lemma = seperator.join(lemmatized_list) 
        return lemma

In [29]:
train['lemma'] = train['clean_plot'].apply(lambda x: lemma(x))
test['lemma'] = test['clean_plot'].apply(lambda x: lemma(x))

## 3. Variables preparation 

In [30]:

X_train = train['lemma']
X_test = test['lemma']    

### 3.1 Target variable one hot encoding

In [31]:
#apply the onehot transformation for the genres vector
y_train = train['genres']
one_hot = MultiLabelBinarizer() # encoder for the  tags 
y_onehot = one_hot.fit_transform(y_train.str.split(' ')) 
y_bin = pd.DataFrame(y_onehot, columns=one_hot.classes_ ) # transform it to Pandas object

In [32]:
# tokenize
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1 # Tambahan vocab_size

In [33]:
#fix max comment lenght 
maxlen = 300 # Diubah dari 100 jadi 500
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen, padding='post') # Ditambah padding post
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen, padding='post') # Ditambah padding post

## 4.The Model

In [34]:
#initialize parameters
inp = Input(shape=(maxlen, )) #maxlen defined earlier
embed_size = 64

In [35]:
# Neural network backbone
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=maxlen)) 
model.add(LSTM(64, return_sequences=True,name='lstm_layer'))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(50, activation="relu"))
model.add(Dense(len(y_bin.columns), activation="softmax"))

In [36]:
# build the model

model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [37]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
tensorboard = TensorBoard(log_dir="logs")
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)

# train the model
batch_size = 16 # Diubah dari 16 ke 32
epochs = 5 # Diubah dari 3 ke 100
model.fit(X_t,y_onehot, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[tensorboard, early_stopping])


Epoch 1/5
1515/2055 [=====================>........] - ETA: 2:56 - loss: 0.2719 - accuracy: 0.3105

KeyboardInterrupt: 

In [ ]:
filename = f'model-{epochs}-{batch_size}.h5'
model.save(filename)

In [ ]:
from keras.models import load_model
model = load_model(filename)
y_pred = model.predict(X_te, batch_size=batch_size, verbose=1)
print(y_pred.shape)

329/329 [==============================] - 14s 38ms/step
(5250, 19)


In [ ]:
df_probs_all = pd.DataFrame(y_pred,columns=y_bin.columns)
df_probs_all.head()

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.173877,0.086269,0.040136,0.012943,0.045592,0.006468,0.010886,0.029080,0.040721,2.395882e-04,0.111999,0.009078,0.002045,0.014095,0.006675,0.327119,0.078986,0.002135,1.656964e-03
1,0.014322,0.007620,0.004599,0.001479,0.019459,0.001785,0.867856,0.048125,0.002308,8.672404e-06,0.001847,0.001029,0.005988,0.001245,0.002069,0.006542,0.004928,0.008778,1.338008e-05
2,0.000300,0.000342,0.000645,0.000384,0.006362,0.000429,0.950058,0.030969,0.000203,5.587655e-07,0.000129,0.000030,0.006805,0.000306,0.000714,0.000096,0.000508,0.001719,3.288493e-07
3,0.058851,0.027527,0.014035,0.005299,0.037687,0.004130,0.008341,0.050772,0.035187,2.270772e-04,0.456656,0.003594,0.000636,0.017664,0.004505,0.160325,0.112036,0.002240,2.884379e-04
4,0.062202,0.005357,0.000544,0.000788,0.049451,0.374309,0.000619,0.169526,0.001205,1.865563e-02,0.008799,0.000111,0.000778,0.060811,0.017564,0.000952,0.225037,0.000269,3.023763e-03


In [ ]:
def top_5_predictions(df):
    N = 5
    cols = df.columns[:-1].tolist()
    a = df[cols].to_numpy().argsort()[:, :-N-1:-1]
    c = np.array(cols)[a]
    d = df[cols].to_numpy()[np.arange(a.shape[0])[:, None], a]
    df1 = pd.DataFrame(c).rename(columns=lambda x : f'max_{x+1}_col')

    predicted_genres = df1["max_1_col"] + ', ' + df1["max_2_col"]+ ', ' +df1["max_3_col"]+ ', ' + df1["max_4_col"]+ ', '+df1["max_5_col"]
    return predicted_genres

In [ ]:
pred_gen = top_5_predictions(df_probs_all)

In [ ]:
submission = pd.DataFrame(data= {'movie_id':test.movie_id,'predicted_genres':pred_gen})

In [ ]:
submission.head()

,movie_id,predicted_genres
0,10827,"Sci-Fi, Action, Horror, Adventure, Thriller"
1,51768,"Documentary, Drama, Comedy, Action, War"
2,36064,"Documentary, Drama, Musical, Comedy, War"
3,33763,"Horror, Sci-Fi, Thriller, Action, Drama"
4,1146,"Crime, Thriller, Drama, Action, Mystery"


In [ ]:
submission.to_csv('submission.csv',index=False)

In [ ]:
new_synopsis_input = [input("Input the synopsis")]
synopsis = new_synopsis_input
new_synopsis_input = tokenizer.texts_to_sequences(new_synopsis_input)
new_synopsis_input = pad_sequences(new_synopsis_input, padding='post', maxlen=maxlen)
predict = model.predict(new_synopsis_input, batch_size=batch_size, verbose=1)
print(predict.shape)

result = pd.DataFrame(predict,columns=y_bin.columns)
print("The synopsis you entered is :\n", synopsis)
print("Genres : ", top_5_predictions(result))

1/1 [==============================] - 0s 47ms/step
(1, 19)
The synopsis you entered is :
 ['asdf']
Genres :  0    Drama, Comedy, Romance, Thriller, Crime
dtype: object
